In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 8
fig_height = 5
fig_format = 'png'
fig_dpi = 200
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/development/projects/active/HIRT/docs/theory':
  os.chdir(r'/development/projects/active/HIRT/docs/theory')

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/usr/lib/python3.12/importlib/_bootstrap.py": 1767871850.0, "/usr/lib/python3.12/importlib/_bootstrap_external.py": 1767871850.0, "/usr/lib/python3.12/zipimport.py": 1767871850.0, "/usr/lib/python3.12/codecs.py": 1767871850.0, "/usr/lib/python3.12/encodings/aliases.py": 1767871850.0, "/usr/lib/python3.12/encodings/__init__.py": 1767871850.0, "/usr/lib/python3.12/encodings/utf_8.py": 1767871850.0, "/usr/lib/python3.12/abc.py": 1767871850.0, "/usr/lib/python3.12/io.py": 1767871850.0, "/usr/lib/python3.12/stat.py": 1767871850.0, "/usr/lib/python3.12/_collections_abc.py": 1767871850.0, "/usr/lib/python3.12/genericpath.py": 1767871850.0, "/usr/lib/python3.12/posixpath.py": 1767871850.0, "/usr/lib/python3.12/os.py": 1767871850.0, "/usr/lib/python3.12/_sitebuiltins.py": 1767871850.0, "/usr/lib/python3/dist-packages/_distutils_hack/__init__.py": 1692360458.0, "/usr/lib/python3/dist-packages/apport_python_hook.py": 1713443495.0, "/usr/lib/python3.12/sitecustomize.py": 1712727227.0, "/usr/lib/

In [2]:
#| label: fig-tx-rx-waveforms
#| fig-cap: TX and RX waveform comparison at 10 kHz. The transmitted signal (top) induces a received signal (bottom) that is both attenuated and phase-shifted. The amplitude ratio and phase lag encode information about the conductivity and geometry of subsurface targets along the TX-RX path.

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_tx_rx_waveforms
import matplotlib.pyplot as plt
from PIL import Image

buf = create_tx_rx_waveforms()
img = Image.open(buf)
plt.figure(figsize=(9, 5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/3100428559.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [3]:
#| label: fig-skin-depth
#| fig-cap: 'Electromagnetic skin depth versus frequency for five soil types spanning the conductivity range from dry sand (0.001 S/m) to saline/contaminated soil (1.0 S/m). The green shaded region indicates the HIRT operating range (2-50 kHz). The dashed line marks 10 kHz, a typical operating frequency.'

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_skin_depth_plot

buf = create_skin_depth_plot()
img = Image.open(buf)
plt.figure(figsize=(9, 5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/3612596633.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [4]:
#| label: fig-coupling-decay
#| fig-cap: Magnetic coupling decay versus distance. The MIT near-field coupling (1/r^3) decays much faster than ERT geometric spreading (1/r^2). The green shaded region indicates typical HIRT probe spacing (1.5-3.5 m). Signal strength drops below practical detection thresholds beyond ~4-5 m separation.

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_coupling_decay_plot

buf = create_coupling_decay_plot()
img = Image.open(buf)
plt.figure(figsize=(8, 5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/2750294519.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
#| label: fig-ert-geometric
#| fig-cap: 'ERT Wenner array configuration showing current injection electrodes (C1, C2) and potential measurement electrodes (P1, P2). Red curves indicate current flow lines; blue dashed lines show equipotential surfaces. The green ellipse marks the zone of maximum sensitivity. The geometric factor K = 2*pi*a for equal electrode spacing a.'

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_ert_geometric_factor

buf = create_ert_geometric_factor()
img = Image.open(buf)
plt.figure(figsize=(9, 4))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/2735051489.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
#| label: fig-frequency-response
#| fig-cap: 'Multi-frequency target discrimination. Metal targets (red) show strong response at lower frequencies with characteristic roll-off. Soil disturbances (purple, dashed) exhibit broader frequency response. The combined signal (gray, dotted) can be decomposed through multi-frequency analysis. The green region indicates the HIRT sweep range (2-50 kHz).'

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_frequency_response

buf = create_frequency_response()
img = Image.open(buf)
plt.figure(figsize=(9, 5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/4123847008.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
#| label: fig-ray-path-comparison
#| fig-cap: 'Comparison of (a) surface and (b) crosshole ray path geometries. Surface sensors create curved, indirect paths with poor sensitivity at depth. Crosshole probes provide direct, straight paths through the target volume. The checkmark in (b) indicates confident target detection; the question mark in (a) indicates ambiguous response at depth.'

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_ray_path_comparison

buf = create_ray_path_comparison()
img = Image.open(buf)
plt.figure(figsize=(10, 4.5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/3828493404.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
#| label: fig-resolution-depth
#| fig-cap: Spatial resolution versus investigation depth. Surface methods (dashed gray) show quadratic degradation of resolution with depth. HIRT crosshole methods (solid green) maintain approximately linear resolution scaling. The red dashed line indicates typical UXB diameter (~0.5 m). The orange shaded region marks typical UXB burial depths (2-4 m).

import sys; sys.path.insert(0, '..') if '..' not in sys.path else None
from diagrams.physics import create_resolution_depth_plot

buf = create_resolution_depth_plot()
img = Image.open(buf)
plt.figure(figsize=(8, 5))
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

/tmp/ipykernel_2207210/1363290330.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
